In [32]:
import pandas as pd
import numpy as np

terridata = pd.read_csv('TerriData_Dim4.csv',encoding='utf-8')

In [33]:
terridata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291756 entries, 0 to 291755
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Código Departamento  291756 non-null  int64  
 1   Departamento         291756 non-null  object 
 2   Código Entidad       291756 non-null  int64  
 3   Entidad              291756 non-null  object 
 4   Dimensión            291756 non-null  object 
 5   Subcategoría         291756 non-null  object 
 6   Indicador            291756 non-null  object 
 7   Dato Numérico        269714 non-null  object 
 8   Dato Cualitativo     0 non-null       float64
 9   Año                  291756 non-null  int64  
 10  Mes                  291756 non-null  int64  
 11  Fuente               291756 non-null  object 
 12  Unidad de Medida     291756 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 28.9+ MB


In [34]:

def transform_datatypes(df: pd.DataFrame) -> pd.DataFrame:
    df["Departamento"]=df["Departamento"].astype('str') 
    df["Entidad"]=df["Entidad"].astype('str')
    df["Dimensión"]=df["Dimensión"].astype('str')
    df["Subcategoría"]=df["Subcategoría"].astype('str') 
    df["Indicador"]=df["Indicador"].astype('str')
    df["Dato Numérico"] = df["Dato Numérico"].apply(lambda x: float(str(x).replace(",", ".")) if x is not None else None)
    df["Dato Cualitativo"]=df["Dato Cualitativo"].astype('str')
    df["Fuente"]=df["Fuente"].astype('str')
    df["Unidad de Medida"]=df["Unidad de Medida"].astype('str') 
    return df

terridata = transform_datatypes(terridata)

In [35]:
terridata.head()

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,Educación,Acceso a la educación,Cobertura bruta en transición,95.42,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
1,5,Antioquia,5000,Antioquia,Educación,Acceso a la educación,Cobertura bruta en transición,78.62,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
2,5,Antioquia,5001,Medellín,Educación,Acceso a la educación,Cobertura bruta en transición,103.23,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
3,5,Antioquia,5002,Abejorral,Educación,Acceso a la educación,Cobertura bruta en transición,51.80,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
4,5,Antioquia,5004,Abriaquí,Educación,Acceso a la educación,Cobertura bruta en transición,96.00,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)


In [36]:
data_departamental = terridata.loc[np.where(terridata['Departamento'] == terridata["Entidad"])]

In [37]:
with open('data_departamental.json', 'w', encoding='utf-8') as file:
    data_departamental.to_json(file, orient='records',default_handler=str,force_ascii=False)

In [38]:
json_departamental = pd.read_json('data_departamental.json',encoding='utf-8')
json_departamental = transform_datatypes(json_departamental)
json_departamental.head()

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
0,1,Colombia,1001,Colombia,Educación,Acceso a la educación,Cobertura bruta en transición,95.42,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
1,5,Antioquia,5000,Antioquia,Educación,Acceso a la educación,Cobertura bruta en transición,78.62,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
2,8,Atlántico,8000,Atlántico,Educación,Acceso a la educación,Cobertura bruta en transición,98.42,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
3,11,Bogotá,11001,Bogotá,Educación,Acceso a la educación,Cobertura bruta en transición,90.34,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)
4,13,Bolívar,13000,Bolívar,Educación,Acceso a la educación,Cobertura bruta en transición,122.88,nan,2005,12,Ministerio de Educación Nacional,Porcentaje (el valor está multiplicado por 100)


In [39]:
indicadores = ["Cobertura bruta en educación - Total",
               "Cobertura neta en educación - Total"]

datos_cobertura_bruta_neta = json_departamental.loc[np.where(
    json_departamental['Indicador'].isin(indicadores))]

keys_to_keep = ["Código Departamento", "Departamento",
                "Indicador", "Año", "Dato Numérico", "Unidad de Medida"]

for year in list(datos_cobertura_bruta_neta.keys()):
    if year not in keys_to_keep:
        del datos_cobertura_bruta_neta[year]


anios = datos_cobertura_bruta_neta["Año"].unique()

differenceMap = {anio: {row["Código Departamento"]: {
    "Departamento": row["Departamento"],
    "Año": row["Año"],
    "Dato Numérico": row["Dato Numérico"],
    "Unidad de Medida": row["Unidad de Medida"]

} for row in datos_cobertura_bruta_neta.to_dict(orient="index", into=dict).values()} for anio in anios}


for row in datos_cobertura_bruta_neta.to_dict(orient="index", into=dict).values():
    if row["Indicador"] == "Cobertura bruta en educación - Total":
        differenceMap[row["Año"]][row["Código Departamento"]
                                  ]["Cobertura bruta en educación - Total"] = row["Dato Numérico"]
    else:
        differenceMap[row["Año"]][row["Código Departamento"]
                                  ]["Cobertura neta en educación - Total"] = row["Dato Numérico"]

min_diff = float("inf")
max_diff = float("-inf")

for year in differenceMap:
    for department in differenceMap[year]:
        resta = differenceMap[year][department]["Cobertura bruta en educación - Total"] - differenceMap[year][department]["Cobertura neta en educación - Total"]
        if resta < min_diff:
            min_diff = resta
        if resta > max_diff:
            max_diff = resta
        differenceMap[year][department]["Diferencia"] =  resta 

datos_cobertura_bruta_neta = differenceMap



In [40]:
import json


def converter(obj):
    if isinstance(obj, np.integer | np.int64 | np.int32 | np.int16 | np.int8):
        return int(obj)
    elif isinstance(obj, np.floating | np.float64 | np.float32 | np.float16):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, np.str_):
        return str(obj)
    elif isinstance(obj, np.void):
        return None
    return obj


convertedTypes = {converter(year): {converter(department): {key: converter(value) for key, value in datos_cobertura_bruta_neta[year][department].items(
)} for department in datos_cobertura_bruta_neta[year]} for year in datos_cobertura_bruta_neta}

convertedTypes["min"] = min_diff
convertedTypes["max"] = max_diff


In [41]:

with open("datos_cobertura_bruta_neta.json", 'w', encoding='utf-8') as file:
    json.dump(convertedTypes, file, ensure_ascii=False, indent=4)
    


In [42]:
year = 2020

with open(f"datos_cobertura_bruta_neta_{year}.json", 'w', encoding='utf-8') as file:
    json.dump(convertedTypes[year], file, ensure_ascii=False, indent=4)
